In [4]:
import pandas as pd
from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet['A1'] = '미리보기'
sheet['B1'] = '분류'

def remove_duplicate_text(dataframe):
    dataframe.drop_duplicates(subset=['미리보기'], inplace=True)
    dataframe.reset_index(drop=True, inplace=True)
    return dataframe

def del_error_next(dataframe):
    drop_indexes = []
    for i in range(len(dataframe)):
        if '# NAME ?' in str(dataframe['미리보기'][i]) or '#NAME?' in str(dataframe['미리보기'][i]):
            drop_indexes.append(i)
    dataframe.drop(drop_indexes, inplace=True)
    dataframe.reset_index(drop=True, inplace=True)
    return dataframe

def check_whats_labels(sentence, dictionary):
    sentence = str(sentence)
    match_counts = {}
    for key in dictionary:
        match_count = 0
        for word in dictionary[key]:
            if word in sentence:
                match_count += 1
        if match_count >= 4:
            match_counts[key] = match_count
    max_count = max(match_counts.values()) if match_counts else None
    max_keys = [key for key, count in match_counts.items() if count == max_count] if max_count else None
    return '종합' if max_keys and len(max_keys) > 5 else max_keys[0] if max_keys else 'NonV'

list_previews = del_error_next(remove_duplicate_text(pd.read_csv('2020.csv')))
list_previews = list_previews['미리보기'].tolist()

dict_labels = pd.read_csv('S:\BD_ai_test\\news_data\crime_labels.csv').to_dict()

crime_labels = {}
for key, value_dict in dict_labels.items():
    crime_labels[key] = list(value_dict.values())[:-1]
    
row_num = 2
for i in range(len(list_previews)):
    if check_whats_labels(list_previews[i], crime_labels) not in ['NonV', '종합']:
        sheet.cell(row=row_num, column=1).value = list_previews[i]
        sheet.cell(row=row_num, column=1).value = list_previews[i]
        sheet.cell(row=row_num, column=2).value = check_whats_labels(list_previews[i],crime_labels)
        row_num += 1



"""for i in range(len(list_previews)):
    sheet.cell(row=i+2, column=1).value = list_previews[i]
    sheet.cell(row=i+2, column=2).value = check_whats_labels(list_previews[i],crime_labels)"""
    
name = input('저장할 파일명을 입력하세요: ')
workbook.save(filename="dst_data\\"+name+".csv")